In [39]:
import os
import json
from PIL import Image

import os
import json
from PIL import Image
def pad_and_crop_image_and_update_labels(image_path, label_path, output_image_path, output_label_path, desired_height=1024, desired_width=1024):
    # Open the image
    image = Image.open(image_path)
    original_width, original_height = image.size

    # Read the label data
    with open(label_path, 'r') as label_file:
        labels = json.load(label_file)

    # Check if image needs cropping or padding
    if original_width > desired_width or original_height > desired_height:
        # Crop the image
        new_image = image.crop((0, 0, min(desired_width, original_width), min(desired_height, original_height)))

        # Filter out labels that fall outside the desired dimensions
        labels = [label for label in labels if label['x'] < desired_width and label['y'] < desired_height]
    else:
        # Pad the image
        new_image = Image.new("RGB", (max(desired_width, original_width), max(desired_height, original_height)), (255, 255, 255))
        new_image.paste(image, (0, 0))

    # Save the new image
    new_image.save(output_image_path)

    # Save updated labels
    with open(output_label_path, 'w') as label_file:
        json.dump(labels, label_file, indent=4)

    print(f"Processed {image_path}; saved new image to {output_image_path} and labels to {output_label_path}")

# Directory containing images and labels
image_directory = "./TMA_WSI_PNGs"
label_directory = "TMA_WSI_Labels_old"

# image_directory = "./augmented_images"
# label_directory = "./augmented_labels"


output_image_directory = "./augmented_images_updated"
output_label_directory = "./augmented_labels_updated"

# Make sure the output directories exist
os.makedirs(output_image_directory, exist_ok=True)
os.makedirs(output_label_directory, exist_ok=True)

# Iterate over all files in the directory
for filename in os.listdir(image_directory):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
        image_path = os.path.join(image_directory, filename)
        label_path = os.path.join(label_directory, os.path.splitext(filename)[0] + '.json')
        output_image_path = os.path.join(output_image_directory, filename)
        output_label_path = os.path.join(output_label_directory, os.path.splitext(filename)[0] + '.json')

        if os.path.exists(label_path):
            pad_and_crop_image_and_update_labels(image_path, label_path, output_image_path, output_label_path)
        else:
            print(f"No label found for {filename}, skipping.")

Processed ./augmented_images\158867_aug_0.png; saved new image to ./augmented_images_updated\158867_aug_0.png and labels to ./augmented_labels_updated\158867_aug_0.json
Processed ./augmented_images\158867_aug_1.png; saved new image to ./augmented_images_updated\158867_aug_1.png and labels to ./augmented_labels_updated\158867_aug_1.json
Processed ./augmented_images\158867_aug_2.png; saved new image to ./augmented_images_updated\158867_aug_2.png and labels to ./augmented_labels_updated\158867_aug_2.json
Processed ./augmented_images\158867_aug_3.png; saved new image to ./augmented_images_updated\158867_aug_3.png and labels to ./augmented_labels_updated\158867_aug_3.json
Processed ./augmented_images\158867_aug_4.png; saved new image to ./augmented_images_updated\158867_aug_4.png and labels to ./augmented_labels_updated\158867_aug_4.json
Processed ./augmented_images\158867_aug_5.png; saved new image to ./augmented_images_updated\158867_aug_5.png and labels to ./augmented_labels_updated\1588